In [1]:
import pandas as pd
import pyvo as vo
import math
import numpy as np 
import astropy.units as u

from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord



In [2]:
# Access SIMBAD db
service = vo.dal.TAPService('http://simbad.u-strasbg.fr/simbad/sim-tap')

In [6]:
# Obtain coords of objects/regions of interest

target_name1 = 'TW Hya'
target_name2 = 'Upper Sco'

target_c1 = SkyCoord.from_name(target_name1)

target_c2 = SkyCoord.from_name(target_name2)

print(target_name1 + ': ' + str(target_c1) + '\n' *2 + target_name2 + ': ' + str(target_c2))

TW Hya: <SkyCoord (ICRS): (ra, dec) in deg
    (165.46627262, -34.70473099)>

Upper Sco: <SkyCoord (ICRS): (ra, dec) in deg
    (243., -23.4)>


In [7]:
# Perform a cone search of TTauri stars around TW Hya w/ Vmag <= 12 

resultset1 = service.search("SELECT oid, main_id, ra, dec, I, B, V, otype FROM basic \
JOIN allfluxes ON oid=allfluxes.oidref \
WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS',165.47, -34.7, 10))=1 \
AND V<= 12 AND otype = 'TT*'\
 ;" 
) 

resultset2 = service.search("SELECT oid, main_id, ra, dec, I, B, V, otype FROM basic \
JOIN allfluxes ON oid=allfluxes.oidref \
WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS',243, -23.4, 20))=1 \
AND V<= 12 AND otype = 'TT*'\
 ;" 
) 

In [8]:
#convert to df and round the results
resultset1 = resultset1.to_table().to_pandas()
resultset2 = resultset2.to_table().to_pandas()

cols = ['ra', 'dec', 'I', 'B', 'V']
resultset1[cols] = resultset1[cols].round(3)
resultset2[cols] = resultset2[cols].round(3)

#drop o-type column
resultset1 = resultset1.drop(columns=['oid', 'otype'])
resultset2 = resultset2.drop(columns=['oid', 'otype'])



In [9]:
# sort and save to csv
sorted_results1 = resultset1.sort_values(by=['dec'], ascending=True)
sorted_results1['main_id']=sorted_results1['main_id'].apply(lambda s: s.decode('utf-8'))
sorted_results1.to_csv('TW Hydrae Assoc.csv', index=False, encoding='utf-8')

sorted_results2 = resultset2.sort_values(by=['dec'], ascending=True)
sorted_results2['main_id']=sorted_results2['main_id'].apply(lambda s: s.decode('utf-8'))
sorted_results2.to_csv('U Sco Assoc.csv', index=False, encoding='utf-8')
